In [1]:
import torch, torch.nn.functional as F
from torch import ByteTensor, DoubleTensor, FloatTensor, HalfTensor, LongTensor, ShortTensor, Tensor
from torch import nn, optim, as_tensor
from torch.utils.data import BatchSampler, DataLoader, Dataset, Sampler, TensorDataset
from torch.nn.utils import weight_norm

In [2]:
import numpy as np
import pandas as pd
import sklearn.model_selection
from tqdm import tqdm
import sys

In [3]:
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
users = ratings.userId.unique()
n_users = len(users)
user_ids = {user: i for i, user in enumerate(users)}
n_users

610

In [5]:
movies = ratings.movieId.unique()
n_movies = len(movies)
movie_ids = {user: i for i, user in enumerate(movies)}
n_movies

9724

In [6]:
train_ratings, test_ratings = sklearn.model_selection.train_test_split(ratings.drop('timestamp', axis=1))

In [7]:
train_ratings.shape, test_ratings.shape

((75627, 3), (25209, 3))

In [8]:
train = torch.tensor(train_ratings.values)
train_target = torch.tensor(train_ratings.rating.values.astype(np.float32))
train_dataset = torch.utils.data.TensorDataset(train, train_target)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64)

In [9]:
test = torch.tensor(test_ratings.values)
test_target = torch.tensor(test_ratings.rating.values.astype(np.float32))

In [10]:
class Model(nn.Module):
    def __init__(self, n_factors):
        super().__init__()
        self.u = nn.Embedding(n_users, n_factors)
        self.m = nn.Embedding(n_movies, n_factors)
        self.u.weight.data.uniform_(0,0.05)
        self.m.weight.data.uniform_(0,0.05)
    
    def forward(self, data):
        users, movies = data[:, 0], data[:, 1]
        u = self.u(torch.tensor([user_ids[user.item()] for user in users]))
        m = self.m(torch.tensor([movie_ids[movie.item()] for movie in movies]))
        return (u * m).sum(1)

In [11]:
model = Model(n_factors=50)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), 1e-1, weight_decay=1e-5, momentum=0.9)

In [12]:
epochs = 3

for epoch in range(1, epochs + 1):

    print(f'epoch: {epoch}')
    
    for i, (data, labels) in enumerate(train_dataloader, 1):
        output = model(data)
        optimizer.zero_grad()
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
    train_mse = round(criterion(model(train), train_target).item(), 2)
    print(f'  train_mse: {train_mse}')
    
    test_mse = round(criterion(model(test), test_target).item(), 2)
    print(f'  test_mse: {test_mse}')

    print()

epoch: 1
  train_mse: 1.25
  test_mse: 1.65

epoch: 2
  train_mse: 0.86
  test_mse: 1.31

epoch: 3
  train_mse: 0.74
  test_mse: 1.24

epoch: 4
  train_mse: 0.66
  test_mse: 1.21

epoch: 5
  train_mse: 0.56
  test_mse: 1.19

epoch: 6
  train_mse: 0.46
  test_mse: 1.18

epoch: 7
  train_mse: 0.37
  test_mse: 1.19

epoch: 8
  train_mse: 0.31
  test_mse: 1.2

epoch: 9
  train_mse: 0.25
  test_mse: 1.22

epoch: 10
  train_mse: 0.21
  test_mse: 1.24

epoch: 11
  train_mse: 0.18
  test_mse: 1.26

epoch: 12
  train_mse: 0.15
  test_mse: 1.28

epoch: 13
  train_mse: 0.13
  test_mse: 1.3

epoch: 14
  train_mse: 0.12
  test_mse: 1.32

epoch: 15
  train_mse: 0.1
  test_mse: 1.33

epoch: 16
  train_mse: 0.09
  test_mse: 1.35

epoch: 17
  train_mse: 0.08
  test_mse: 1.36

epoch: 18
  train_mse: 0.08
  test_mse: 1.38

epoch: 19
  train_mse: 0.07
  test_mse: 1.39

epoch: 20
  train_mse: 0.07
  test_mse: 1.4

